### installation guide
First I needed the postgresql database from musicbrainz. It turned out the documentation for setting up the server was quite a bit behind. However, after some trial and error, I found a good way to set up the database:

follow [the musicbrainz documentation](https://musicbrainz.org/doc/MusicBrainz_Server/Setup) until accessing the database.

To get access to the virtual machine, log in (or ssh `ssh -p 2222 vagrant@localhost`) with username `vagrant` and password `vagrant`.

Docker is not configured to expose the database by default. To change the configuration, diff the following lines to `musicbrainz/musicbrainz-docker/docker-compose.yml`

```
14,15d13
<     ports:
<       - "5432:5432"
20a19,20
>     expose:
>       - "5432"
```

Now, in virtualbox, port forward (settings -> network -> advanced -> port forwarding) port 5432. This enables you to connect to the postgres database from your host machine.

After this, you can connect with user `musicbrainz` with password `musicbrainz` on your host machine (for example with `psql -h localhost -U musicbrainz -p 5432`)

In [1]:
import psycopg2

conn = psycopg2.connect(user="musicbrainz", password="musicbrainz")

One of the main early objectives of this project is to get all the aliases for a given composer (or other type of artist, since musicbrainz does not destinguish between the two).

The function below shows a simple implementation of this.

In [2]:
def get_aliases(person):
    '''
    Gets all the aliases for a given person by querying both the artist_credit_name
    table and the artist_alias table. The function throws away duplicates by returning
    the result as a set.
    '''
    with conn:
        with conn.cursor() as curs:
            curs.execute(
                """
                select artist_credit_name.name 
                from artist
                left join artist_credit_name on artist_credit_name.artist = artist.id 
                where artist.name = %s
                """, [person]
            )
            credit_names = curs.fetchall()
            credit_names = {name[0] for name in credit_names}

            curs.execute(
                """select artist_alias.name 
                from artist
                left join artist_alias on artist.id = artist_alias.artist 
                where artist.name = %s""", [person]
            )

            aliases = curs.fetchall()
            aliases = {name[0] for name in aliases}

            return credit_names | aliases

get_aliases('Alexander von Zemlinsky')
        
    

{'Alexander Zemlinsky',
 'Alexander von Zemlinsky',
 'Zemlinksy',
 'Zemlinsky',
 'ツェムリンスキー'}

As you can see, the list of aliases contains multiple different spellings of Zemlinsky.



In [3]:
get_aliases('Wolfgang Amadeus Mozart')

{'English Classical Players',
 'Johannes Chrysostomus Wolfgangus Theophilus Mozart',
 'Mosart',
 'Mozart',
 'Volfgangs Amadejs Mocarts',
 'W A Mozart',
 'W. A. Mozart',
 'W. A. Mozart/Arr. by Nova',
 'W. Amadeus Mozart',
 'W.-A. Mozart',
 'W.A Mozart',
 'W.A. Mozart',
 'W.A.Mozart',
 'WA Mozart',
 'WA. Mozart',
 'Wolfang A. Mozart',
 'Wolfang Amadeus Mozart',
 'Wolfgan Amadeus Mozart',
 'Wolfgang A. Mozart',
 'Wolfgang Amade Mozart',
 'Wolfgang Amadeaus Mozart',
 'Wolfgang Amadeus Mozart',
 'Wolfgang Amadeus Mozart (1756-1791)',
 'Wolfgang Amadues Mozart',
 'Wolfgang Amedeus Mozart',
 'Wolfgang Anadeus Mozart',
 'Wolfgang Armadeus Mozart',
 'Wolfgang Mozart',
 'mozart',
 'Вольфганг Амадей Моцарт',
 'Моцарт',
 'モーツァルト',
 'ヴォルフガンク・アマデウス・モーツァルト',
 '볼프강 아마데우스 모짜르트'}

Using a more well known composer as input gives even more useful results.

### Creating a searchable database from these queries
De volgende stap is het verzamelen van deze data voor alle artiesten in de database en die vervolgens doorzoekbaar te maken. Cynthia stelde voor om de aliases om te zetten naar n-grams (als ik het goed heb begrepen). Deze resultaten zijn over het algemeen echter zo kort dat het mij persoonlijk beter lijkt om eerst een simpelere aanpak te gebruiken en de aliases allemaal bij elkaar te halen en vervolgens te tokenizen (op spaties, punten en een aantal andere leestekens, zoals '/'). Vervolgens zijn er voor iedere persoon dus een aantal woorden die daarmee corresponderen. Die woorden worden de zoektermen in de database. Oftewel, een [bag of words met Tf-idf (term frequency, inverse document frequency)](https://nlp.stanford.edu/IR-book/html/htmledition/term-frequency-and-weighting-1.html). 

Vervolgens kan de artiesten zoek database makkelijk worden uitgebreid met de lijst van werken en de lijst van cd's in de database. Dezelfde bag of words kan namelijk ook worden toegepast op cd's en werken. Waar iedere afzonderlijke bag een lijst is met de naam en alias van de componist, de uitvoerders en alle relevante informatie van het werk en de cd zelf. Ik verwacht dat je daarmee al heel veel relevante data kan krijgen. Het is dan zelfs mogelijk om bijvoorbeeld de metadata van de componist zwaarder te laten wegen dan de metadata van de cd of omgekeerd via [weighted zone scoring](https://nlp.stanford.edu/IR-book/html/htmledition/weighted-zone-scoring-1.html). 

De volgende stap is spelling correctie op de individuele termen. Als dit is geimplementeerd en de resultaten nog niet relevant genoeg zijn kunnen we ook nog de positie van de zoektermen meenemen via n-grams.

Ik hoop dat je dit plan van aanpak een goed idee vindt!